<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/git_teste_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
#!rm -rf cassandra

In [3]:
# clona o repositorio
!git clone https://github.com/apache/cassandra.git

Cloning into 'cassandra'...
remote: Enumerating objects: 381182, done.
remote: Counting objects: 100% (314/314), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 381182 (delta 71), reused 200 (delta 35), pack-reused 380868
Receiving objects: 100% (381182/381182), 382.22 MiB | 10.18 MiB/s, done.
Resolving deltas: 100% (221822/221822), done.


In [4]:
def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

In [5]:
# Quantidade de arquivos na pasta cassandra
qtd_arquivos = !find cassandra -type f | wc -l

# Quantidade de commits do repositorio cassandra
qtd_commits = !cd cassandra && git log --oneline | grep -c .

# Tamanho (MB) da pasta cassandra
tamanho_pasta_mb = !du -hs --block-size=1M cassandra | cut -f1

# Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto cassandra)
qtd_colaboradores_commits = !cd cassandra && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l

# Data do primeiro commit do projeto cassandra
data_primeiro_commit =!cd cassandra && git log --reverse --pretty=format:%ad --date=short | head -1

# Data do ultimo commit do projeto cassandra
data_ultimo_commit = !cd cassandra && git log -1 --pretty=format:%ad --date=short

In [6]:
dict_info_repo = {
  'nome':'cassandra',
	'qtd_arquivos':qtd_arquivos,
	'qtd_commits':qtd_commits,
	'tamanho_pasta_mb':tamanho_pasta_mb,
	'qtd_colaboradores_commits':qtd_colaboradores_commits,
	'data_primeiro_commit':data_primeiro_commit,
	'data_ultimo_commit':data_ultimo_commit,
}

df_info_repo = pd.DataFrame(dict_info_repo)
df_info_repo

,nome,qtd_arquivos,qtd_commits,tamanho_pasta_mb,qtd_colaboradores_commits,data_primeiro_commit,data_ultimo_commit
0,cassandra,5188,28426,487,578,2009-03-02,2023-03-09


In [7]:
!sudo apt-get install cloc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cloc is already the newest version (1.82-1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [8]:
# Detalha os tipos de arquivos que aparecem no repositório do cassandra
relacao_linguagens = !cd cassandra && cloc .

relacao_linguagens[checa_indice_language(relacao_linguagens):]

['Language                      files          blank        comment           code',
 '--------------------------------------------------------------------------------',
 'Java                           3877         116507         146654         613993',
 'YAML                             45            767           2563         330789',
 'AsciiDoc                        109           5592             11          20115',
 'Python                           31           2460           3163           8332',
 'XML                              33            201            478           3740',
 'Markdown                         13            771              0           3391',
 'SVG                              17            197           1399           2937',
 'Bourne Shell                     82            371            988           1984',
 'ANTLR Grammar                     3            267            288           1897',
 'Ant                               3            185            2

In [9]:
linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
linguagens_encontradas

['Java                           3877         116507         146654         613993',
 'YAML                             45            767           2563         330789',
 'AsciiDoc                        109           5592             11          20115',
 'Python                           31           2460           3163           8332',
 'XML                              33            201            478           3740',
 'Markdown                         13            771              0           3391',
 'SVG                              17            197           1399           2937',
 'Bourne Shell                     82            371            988           1984',
 'ANTLR Grammar                     3            267            288           1897',
 'Ant                               3            185            233           1787',
 'Bourne Again Shell                2             23             71            164',
 'Maven                             1             25             

In [10]:
lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()

l_temp = []
vl_temp = []
for linha in linguagens_encontradas:
  elementos = linha.split(' ')
  novos_elementos = []
  linguagem_encontrada = []
  valores_da_linguagem = []
  for item in elementos:
    if item != '':
      novos_elementos.append(item)
  linguagem_encontrada.append(novos_elementos[:-4])
  valores_da_linguagem.append(novos_elementos[-4:])
  print(linguagem_encontrada, valores_da_linguagem)
  l_temp = l_temp + linguagem_encontrada
  vl_temp = vl_temp + valores_da_linguagem

[['Java']] [['3877', '116507', '146654', '613993']]
[['YAML']] [['45', '767', '2563', '330789']]
[['AsciiDoc']] [['109', '5592', '11', '20115']]
[['Python']] [['31', '2460', '3163', '8332']]
[['XML']] [['33', '201', '478', '3740']]
[['Markdown']] [['13', '771', '0', '3391']]
[['SVG']] [['17', '197', '1399', '2937']]
[['Bourne', 'Shell']] [['82', '371', '988', '1984']]
[['ANTLR', 'Grammar']] [['3', '267', '288', '1897']]
[['Ant']] [['3', '185', '233', '1787']]
[['Bourne', 'Again', 'Shell']] [['2', '23', '71', '164']]
[['Maven']] [['1', '25', '20', '108']]
[['make']] [['2', '20', '20', '70']]
[['CSS']] [['1', '14', '20', '69']]
[['DOS', 'Batch']] [['2', '1', '0', '56']]
[['JSON']] [['6', '0', '0', '52']]
[['HTML']] [['1', '104', '519', '50']]
[['diff']] [['3', '16', '44', '40']]
[['reStructuredText']] [['2', '22', '31', '40']]
[[]] [['--------------------------------------------------------------------------------']]
[['SUM:']] [['4233', '127543', '156502', '989614']]
[[]] [['-----------

In [11]:
l_temp = l_temp[:-3]
for i, each in enumerate(l_temp):
  print(i,each)

vl_temp = vl_temp[:-3]
for i,each in enumerate(vl_temp):
  print(i,each)
  lista_files.append(each[0])
  lista_blank.append(each[1]) 
  lista_comment.append(each[2]) 
  lista_code.append(each[3])

0 ['Java']
1 ['YAML']
2 ['AsciiDoc']
3 ['Python']
4 ['XML']
5 ['Markdown']
6 ['SVG']
7 ['Bourne', 'Shell']
8 ['ANTLR', 'Grammar']
9 ['Ant']
10 ['Bourne', 'Again', 'Shell']
11 ['Maven']
12 ['make']
13 ['CSS']
14 ['DOS', 'Batch']
15 ['JSON']
16 ['HTML']
17 ['diff']
18 ['reStructuredText']
0 ['3877', '116507', '146654', '613993']
1 ['45', '767', '2563', '330789']
2 ['109', '5592', '11', '20115']
3 ['31', '2460', '3163', '8332']
4 ['33', '201', '478', '3740']
5 ['13', '771', '0', '3391']
6 ['17', '197', '1399', '2937']
7 ['82', '371', '988', '1984']
8 ['3', '267', '288', '1897']
9 ['3', '185', '233', '1787']
10 ['2', '23', '71', '164']
11 ['1', '25', '20', '108']
12 ['2', '20', '20', '70']
13 ['1', '14', '20', '69']
14 ['2', '1', '0', '56']
15 ['6', '0', '0', '52']
16 ['1', '104', '519', '50']
17 ['3', '16', '44', '40']
18 ['2', '22', '31', '40']


In [12]:
dict_dados_linguagens = {
    'linguagens':l_temp, 
    'files':lista_files, 
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
}

df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
df_dados_linguagens

,linguagens,files,blank,comment,code
0,[Java],3877,116507,146654,613993
1,[YAML],45,767,2563,330789
2,[AsciiDoc],109,5592,11,20115
3,[Python],31,2460,3163,8332
4,[XML],33,201,478,3740
5,[Markdown],13,771,0,3391
6,[SVG],17,197,1399,2937
7,"[Bourne, Shell]",82,371,988,1984
8,"[ANTLR, Grammar]",3,267,288,1897
9,[Ant],3,185,233,1787


In [13]:
df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))	
df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))	
df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))

df_dados_linguagens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   linguagens  19 non-null     object
 1   files       19 non-null     int64 
 2   blank       19 non-null     int64 
 3   comment     19 non-null     int64 
 4   code        19 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 888.0+ bytes
